In [1]:
print("Hello world")

Hello world


In [2]:
from cerberus import Validator

schema = {
    'username': {'type': 'string', 'minlength': 3, 'maxlength': 30},
    'age': {'type': 'integer', 'min': 1},
    'initials': {'type': 'string', 'regex': '^[A-Z]{2}$'},
    'password': {'type': 'string', 'forbidden': ['xy'], 'regex': '@'},
    'email': {'type': 'string', 'regex': r'^\S+@\S+\.\S+$'}
}

v = Validator(schema)

# Beispiel-Daten
data = {
    "username": "user123",
    "age": 25,
    "initials": "AB",
    "password": "myp@ssword",
    "email": "user@example.com"
}

# Validierung
if v.validate(data):
    print("Valid data:", v.document)
else:
    print("Invalid data:", v.errors)

Invalid data: {'password': ["value does not match regex '@'"]}


In [8]:
schema = {
    'name': {'type': 'string'},
    'age': {'type': 'integer', 'min': 1, 'max': 125},
    'fortune': {'type': 'string'},

}


val = Validator(schema)

val.validate({'name': 'John Doe', 'fortune': 7})

print(val.errors)



{'fortune': ['must be of string type']}


In [36]:
import openpyxl
import os

def cell_adjacent_to_keyword(worksheet, keyword, orientation="right"):
    """
    Returns "" if target cell is empty, None if target cell does not exist
    """

    if orientation not in ["right", "left", "up", "down"]:
        raise ValueError("Invalid orientation. Must be one of 'right', 'left', 'up', 'down'")

    return_val = ""

    for row in worksheet.iter_rows():
        for cell in row:
            if cell.value == keyword:
                if orientation == "right":
                    return_val = cell.offset(row=0, column=1).value
                elif orientation == "left":
                    if cell.column == 1:
                        raise ValueError("Cannot get cell to the left of the first cell in a row")
                    return_val = cell.offset(row=0, column=-1).value
                elif orientation == "up":
                    if cell.row == 1:
                        raise ValueError("Cannot get cell above the first row")
                    return_val = cell.offset(row=-1, column=0).value
                elif orientation == "down":
                    return_val = cell.offset(row=1, column=0).value
    
    if return_val is None:
        return ""
    else:
        return return_val


print(os.getcwd())

wb = openpyxl.load_workbook('../testdata/datenerfassung_leer_V1.7.2.xlsx')
ws = wb["Specification"]

print(f"Return:{cell_adjacent_to_keyword(ws, "Version", orientation="left")}|")

/Users/andreas/Library/Mobile Documents/iCloud~md~obsidian/Documents/CBAM/ce_xml_converter/tests/playground


ValueError: Cannot get cell to the left of the first cell in a row

In [43]:
import yaml
from yaml.loader import SafeLoader
from pprint import pprint

layout_dict = None

try:
    with open("../../version_layouts.yml", "r") as f:
        layout_dict = yaml.load(f, Loader=SafeLoader)
except FileNotFoundError:
    print("The specified file was not found.")
except yaml.YAMLError as exc:
    print(f"Error parsing YAML file: {exc}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

if layout_dict:
    pprint(layout_dict)
else:
    print("No data loaded from the YAML file.")

{'base': {'sheets': [{'actual_name': 'Allgemeine_Informationen',
                      'legacy_name': 'Allgemeine_Informationen',
                      'tables': [{'fields': [{'field_actual_name': 'Unternehmen',
                                              'field_legacy_name': 'Unternehmen',
                                              'type': 'string_m'},
                                             {'field_name': 'Straße',
                                              'type': 'string_m'},
                                             {'field_name': 'Hausnummer',
                                              'type': 'string_m'},
                                             {'field_name': 'Adresszusatz',
                                              'type': 'string_m'},
                                             {'field_name': 'Postleitzahl',
                                              'type': 'string_m'},
                                             {'field_name': 'Stadt',
      

In [12]:
import openpyxl
import os
from openpyxl.utils import column_index_from_string, get_column_letter


def keyword_index(worksheet, keyword, fixed_row=None, fixed_column=None):
    """
    Returns the index of row and column of the keyword in the worksheet as a string.

    If fixed_row or fixed_column are provided, the search is limited to that row or column.
    """
    if fixed_row is not None:
        for cell in worksheet[fixed_row]:
            if cell.value == keyword:
                return get_column_letter(cell.column)+ str(cell.row)
    elif fixed_column is not None:
        for cell in worksheet[fixed_column]:
            if cell.value == keyword:
                return get_column_letter(cell.column) + str(cell.row)
    else:
        for row in worksheet.iter_rows():
            for cell in row:
                if cell.value == keyword:
                    return get_column_letter(cell.column) + str(cell.row)

    raise ValueError(f"Keyword '{keyword}' not found in worksheet")

wb = openpyxl.load_workbook('../testdata/datenerfassung_leer_V1.7.2.xlsx')
ws = wb["Specification"]

index_str = keyword_index(ws, "Version")
print(index_str)
print(keyword_index(ws, "1.7.3", fixed_row=''.join(filter(str.isdigit, index_str))))


A3


ValueError: Keyword '1.7.3' not found in worksheet

In [6]:
import re

def col_from_hex(hex_color):
    """Converts a hex color code to an ANSI escape sequence for the terminal."""
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
    return f"\033[38;2;{r};{g};{b}m"

def parse_colored_string(s, color_dict):
    """Parses a string with custom color tags and returns a string with ANSI escape sequences."""
    reset = "\033[0m"
    stack = []
    result = ""

    pattern = re.compile(r"<(\/?[#a-zA-Z0-9]+)>|([^<]+)")

    style_dict = {
        "b": "\033[1m",     # Bold
        "i": "\033[3m",     # Italic
        "u": "\033[4m",     # Underline
        "mark": "\033[7m",  # Highlight (Reverse Video)
    }

    for match in pattern.finditer(s):
        tag, text = match.groups()

        if tag:
            if tag.startswith("/"):
                if tag == "/r":
                    result += reset
                    stack.clear()
                elif stack:
                    stack.pop()
                    result += stack[-1] if stack else reset
            else:
                if tag.startswith("#"):  # Wenn der Tag ein Hex-Code ist
                    color = col_from_hex(tag)
                elif tag in style_dict:  # Wenn es ein Stil-Tag ist
                    color = style_dict[tag]
                else:  # Wenn der Tag ein benannter Farbname ist
                    color = color_dict.get(tag, "")
                stack.append(color)
                result += color
        elif text:
            result += text

    return result + reset

# Beispiel für die Nutzung
color_dict = {
    "red": col_from_hex("#ff5733"),
    "green": col_from_hex("#28a745"),
    "blue": col_from_hex("#007bff"),
}

formatted_string = (
    "Hello, <#ff5733> beautiful </r> world. "
    "Ich habe einen <red> roten <green> grünen </green> Apfel. "
    "<b>Fetter Text</b> </r>und <i>kursiver Text</i>. "
    "Auch <u>unterstrichen</u> und <mark>hervorgehoben</mark>."
)
parsed_string = parse_colored_string(formatted_string, color_dict)
print(parsed_string)

Hello,  beautiful  world. Ich habe einen  roten  grünen  Apfel. Fetter Text und kursiver Text. Auch unterstrichen und hervorgehoben.


In [5]:
import xattr
import plistlib

def get_macos_tags(file_path):
    try:
        # Versuchen, das erweiterte Attribut für Finder-Tags zu lesen
        tags = xattr.getxattr(file_path, 'com.apple.metadata:_kMDItemUserTags')
        # Umwandeln der Binärdaten im plist-Format in eine Python-Struktur
        tags = plistlib.loads(tags)
        return tags
    except (KeyError, OSError, plistlib.InvalidFileException) as e:
        # Wenn keine Tags vorhanden sind oder ein Fehler auftritt, wird eine Nachricht zurückgegeben
        return f"Keine Tags gefunden oder Fehler beim Auslesen: {e}"

# Beispielaufruf für eine Datei
file_path = "/Users/andreas/Lokal/CBAM - Lokal/4 - Berichterstellung/Report Creation Workflow/input/customer_Q3 Datenerfassung zu CBAM-Reporterstellung -.xlsx"  # Den Pfad zur gewünschten Datei hier angeben
get_macos_tags(file_path)

[]